In [1]:
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
from panda_challenge import ClassifcationDatasetMultiCropOneImage
from panda_challenge.train_utils import QWKCallback, get_optimizer, get_scheduler
from panda_challenge import ClassifcationModel
from panda_challenge.models import AdaptiveConcatPool2d
from panda_challenge.utils import freeze, unfreeze

import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import albumentations as A
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR
import torch
from catalyst.contrib.nn.schedulers.onecycle import OneCycleLRWithWarmup
from catalyst.contrib.nn.optimizers import RAdam, Lookahead
from warmup_scheduler import GradualWarmupScheduler
import collections

from sklearn.metrics import cohen_kappa_score

from catalyst.dl.runner import SupervisedRunner
from catalyst.dl.callbacks import CriterionCallback
from catalyst.core.callbacks import EarlyStoppingCallback
from catalyst.core.callbacks import MetricAggregationCallback
from catalyst.core.callbacks import CheckpointCallback

from pytorch_toolbelt.losses import BinaryFocalLoss

import os 

from collections import OrderedDict
from typing import List

from tqdm.auto import tqdm

%matplotlib inline

/home/skolchenko/.local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/skolchenko/.conda/envs/deeplearning/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/skolchenko/.conda/envs/deeplearning/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/skolchenko/.conda/envs/deeplearning/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/skolchenko/.conda/envs/deeplearning/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning:

can't resolve pac

In [2]:
N_SPLITS = 5
N_TILES = 36
IMAGE_SIZE = 256
LEVEL = 1
BATCH_SIZE = 4
NUM_WORKERS = 64
N_EPOCHS = 100
N_FROZEN_ENCODER = 5
WARMUP_EPOCHS = 5
INIT_LR = 3e-4
WARMUP_FACTOR = 1
NUM_CLASSES = 5
IMAGE_FOLDER = '/data/personal_folders/skolchenko/panda/train_images/'
N_TILES_ROW = int(np.sqrt(N_TILES))
MODEL_NAME = 'resnet34'
PATIENCE = 10

In [3]:
transforms_train_individual = A.Compose([
    A.OneOf(
    [
        A.Transpose(p=1.0),
        A.VerticalFlip(p=1.0),
        A.HorizontalFlip(p=1.0),
        A.RandomRotate90(p=1.0),
        A.NoOp()
    ], p=1.0),
    A.OneOf(
    [
        A.ElasticTransform(p=1.0),
        A.GridDistortion(p=1.0),
        A.OpticalDistortion(p=1.0),
        A.NoOp()
    ], p=1.0),
    A.OneOf(
    [
        A.GaussNoise(p=1.0),
        A.GaussianBlur(p=1.0),
        A.ISONoise(p=1.0),
        A.CoarseDropout(p=1.0, max_holes=16, max_height=16, max_width=16),
        A.NoOp()
    ], p=1.0)
])
transforms_train_global = A.Compose([
    A.Normalize(),
    A.OneOf(
    [
        A.Transpose(p=1.0),
        A.VerticalFlip(p=1.0),
        A.HorizontalFlip(p=1.0),
        A.RandomRotate90(p=1.0),
        A.NoOp()
    ], p=1.0),
    A.RandomGridShuffle(grid=(N_TILES_ROW, N_TILES_ROW))
])
transforms_train_global_tta = A.Compose([
    A.OneOf(
    [
        A.Transpose(p=1.0),
        A.VerticalFlip(p=1.0),
        A.HorizontalFlip(p=1.0),
        A.RandomRotate90(p=1.0),
        A.NoOp()
    ], p=1.0),
    A.RandomGridShuffle(grid=(N_TILES_ROW, N_TILES_ROW), p=1.0),
    A.Normalize()
])
transforms_valid_global = A.Compose([
    A.Normalize()
])

In [4]:
data_holdout = pd.read_csv('./pseudolabeled_holdout.csv')
data_train = pd.read_csv('./pseudolabeled_data.csv')
data_train.head()

,image_id,data_provider,isup_grade,gleason_score,fold_idx,class_0,class_1,class_2,class_3,class_4
0,b993648443f3e9ff352b64202528592b,karolinska,1,3+3,0,0.999334,0.538650,0.082998,0.022668,0.001731
1,39f81a4d7b5b86706391dcc09a4c2e94,karolinska,2,3+4,0,0.978157,0.500730,0.211926,0.078533,0.012538
2,a5e22243152d5cc871983364ae7b6e95,karolinska,0,0+0,0,0.340560,0.061330,0.011089,0.003904,0.000478
3,c1f151640dce9fc4e33569700be7d52d,radboud,0,negative,0,0.070296,0.029056,0.020019,0.010508,0.003844
4,c7c18a00f4c61c696c30c721bf7621d2,karolinska,2,3+4,0,0.980977,0.335641,0.049245,0.016449,0.001381


In [5]:
def train_fold(fold_idx):
    criterion = nn.BCEWithLogitsLoss()
    #criterion = nn.MSELoss()
    LOG_DIR = f'/data/personal_folders/skolchenko/panda/logs/{MODEL_NAME}_size{IMAGE_SIZE}_tiles{N_TILES}_level{LEVEL}_heavyHead_fold{fold_idx}_pseudolabeled'
    print(f'Saving to {LOG_DIR}')
    model = ClassifcationModel(model_name=MODEL_NAME, num_classes=NUM_CLASSES, pretrained=True)
    model.head = nn.Sequential(
                AdaptiveConcatPool2d((1, 1)),
                nn.Flatten(),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(2*model.nc, model.nc//2),
                nn.ReLU(),
                nn.Dropout(0.15),                
                nn.Linear(model.nc//2, NUM_CLASSES))
    model.cuda()    
    
    fold_data_train = data_train.loc[data_train['fold_idx']!=fold_idx]
    fold_data_val = data_train.loc[data_train['fold_idx']==fold_idx]    
    
    train_dataset = ClassifcationDatasetMultiCropOneImage(
        fold_data_train, 
        IMAGE_SIZE,
        IMAGE_FOLDER,
        LEVEL,
        N_TILES,
        transform_individual=transforms_train_individual,
        transform_global=transforms_train_global,
        normalize=False,
        load_pickled_tiles=True,
        pickled_tiles_folder=f'/data/personal_folders/skolchenko/panda/pickled_tiled_images_{LEVEL}_{N_TILES}_{IMAGE_SIZE}',
        output_type='ordinal',
        pseudo_labels_columns=[f'class_{x}' for x in range(5)]
        )
    val_dataset = ClassifcationDatasetMultiCropOneImage(
        fold_data_val, 
        IMAGE_SIZE,
        IMAGE_FOLDER,
        LEVEL,
        N_TILES,
        transform_global=transforms_valid_global,
        normalize=False,
        load_pickled_tiles=True,
        pickled_tiles_folder=f'/data/personal_folders/skolchenko/panda/pickled_tiled_images_{LEVEL}_{N_TILES}_{IMAGE_SIZE}',
        output_type='ordinal',
        pseudo_labels_columns=[f'class_{x}' for x in range(5)]
        )
    holdout_dataset = ClassifcationDatasetMultiCropOneImage(
        data_holdout, 
        IMAGE_SIZE,
        IMAGE_FOLDER,
        LEVEL,
        N_TILES,
        transform_global=transforms_valid_global,
        normalize=False,
        pickled_tiles_folder=f'/data/personal_folders/skolchenko/panda/pickled_tiled_images_{LEVEL}_{N_TILES}_{IMAGE_SIZE}',
        load_pickled_tiles=True,       
        output_type='ordinal',
        pseudo_labels_columns=[f'class_{x}' for x in range(5)]
        )    
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)  
    holdout_loader = DataLoader(holdout_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)  
    
    loaders = collections.OrderedDict()
    loaders["train"] = train_loader
    loaders["valid"] = val_loader
    loaders["holdout"] = holdout_loader
    losses = dict({
        'loss_isup': criterion,
    })
    runner = SupervisedRunner(
        input_key='features',
        input_target_key="targets_isup_pseudolabels",
        output_key="logits_isup"
        )    
    
    callbacks = [
        CriterionCallback(
            input_key="targets_isup_pseudolabels",
            output_key="logits_isup",
            prefix="loss_isup",
            criterion_key='loss_isup',
            multiplier=1.0
        ),
        QWKCallback(input_key="targets_isup", 
                    output_key='logits_isup',
                    qwk_name='ordinal'
                    #qwk_name='simple',
                   ),
        MetricAggregationCallback(
            prefix="loss",
            mode="weighted_sum",
            metrics={
                "loss_isup": 1.0
            }
        ),
        EarlyStoppingCallback(patience = PATIENCE, min_delta=1e-4),
        CheckpointCallback(save_n_best = 5)    
    ]    
    
    freeze(model)
    optimizer = RAdam(model.parameters(), lr=INIT_LR)
    optimizer = Lookahead(optimizer)
    runner.train(
        model=model,
        criterion=losses,
        optimizer=optimizer,
        callbacks=callbacks,
        loaders=loaders,
        logdir=LOG_DIR.format(fold_idx),
        main_metric='loss',
        num_epochs=N_FROZEN_ENCODER,
        verbose=True,
        minimize_metric=True,
        fp16=True
    )    
    model.load_state_dict(torch.load(LOG_DIR.format(fold_idx)+'/checkpoints/best.pth')['model_state_dict'])
    
    
    unfreeze(model)
    optimizer = RAdam(model.parameters(), lr=INIT_LR/WARMUP_FACTOR)
    optimizer = Lookahead(optimizer)
    scheduler_cosine = CosineAnnealingLR(
        optimizer,
        N_EPOCHS-WARMUP_EPOCHS)
    scheduler = GradualWarmupScheduler(
        optimizer,
        multiplier=WARMUP_FACTOR, 
        total_epoch=WARMUP_EPOCHS,
        after_scheduler=scheduler_cosine)    
    
    runner.train(
        model=model,
        criterion=losses,
        scheduler=scheduler,
        optimizer=optimizer,
        callbacks=callbacks,
        loaders=loaders,
        logdir=LOG_DIR.format(fold_idx),
        main_metric='loss',
        num_epochs=N_EPOCHS,
        verbose=True,
        minimize_metric=True,
        fp16=True
    )    

In [ ]:
train_fold(0)

Saving to /data/personal_folders/skolchenko/panda/logs/resnet34_size256_tiles36_level1_heavyHead_fold0_pseudolabeled
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
1/5 * Epoch (train):   0% 0/1692 [00:00<?, ?it/s]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
1/5 * Epoch (train):   0% 1/1692 [00:12<5:51:44, 12.48s/it, batch_q

/home/skolchenko/.conda/envs/deeplearning/lib/python3.7/site-packages/sklearn/metrics/_classification.py:604: RuntimeWarning:

invalid value encountered in true_divide



1/5 * Epoch (valid): 100% 423/423 [00:59<00:00,  7.10it/s, batch_qwk=0.000e+00, loss=0.476, loss_isup=0.476] 
1/5 * Epoch (holdout): 100% 374/374 [00:54<00:00,  6.89it/s, batch_qwk=0.000e+00, loss=0.508, loss_isup=0.508] 
[2020-07-17 08:24:36,216] 
1/5 * Epoch 1 (_base): lr=0.0003 | momentum=0.9000
1/5 * Epoch 1 (train): batch_qwk=0.0554 | loss=0.7746 | loss_isup=0.7746 | qwk=0.0411
1/5 * Epoch 1 (valid): batch_qwk=nan | loss=0.5527 | loss_isup=0.5527 | qwk=0.0802
1/5 * Epoch 1 (holdout): batch_qwk=0.0883 | loss=0.5476 | loss_isup=0.5476 | qwk=0.0902
2/5 * Epoch (train):  22% 378/1692 [01:49<05:45,  3.80it/s, batch_qwk=-2.500e-01, loss=0.479, loss_isup=0.479]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
2/5 * Epoch (train): 100% 1692/1692 [07:22<00:00,  3.83it/s, batch_qwk=-2.308e-01, loss=0.617, loss_isup=0.617]
2/5 * Epoch (valid): 100% 423/423 [00:59<00:00,  7.06it/s, batch_qwk=-1.250e-01, loss=0.612, loss_isup=0.612]
2/5 * Epoch (holdout): 100% 374/

/home/skolchenko/.conda/envs/deeplearning/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:113: UserWarning:

Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate



2/100 * Epoch (train):  27% 452/1692 [02:36<06:26,  3.20it/s, batch_qwk=-2.500e-01, loss=0.402, loss_isup=0.402]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
2/100 * Epoch (train): 100% 1692/1692 [09:03<00:00,  3.11it/s, batch_qwk=1.000, loss=0.236, loss_isup=0.236]     
2/100 * Epoch (valid): 100% 423/423 [01:00<00:00,  6.97it/s, batch_qwk=0.200, loss=0.348, loss_isup=0.348]     
2/100 * Epoch (holdout): 100% 374/374 [00:55<00:00,  6.77it/s, batch_qwk=0.000e+00, loss=0.292, loss_isup=0.292] 
[2020-07-17 09:24:00,663] 
2/100 * Epoch 7 (_base): lr=0.0001 | momentum=0.9000
2/100 * Epoch 7 (train): batch_qwk=nan | loss=0.4309 | loss_isup=0.4309 | qwk=0.5747
2/100 * Epoch 7 (valid): batch_qwk=nan | loss=0.3679 | loss_isup=0.3679 | qwk=0.7007
2/100 * Epoch 7 (holdout): batch_qwk=0.5769 | loss=0.3627 | loss_isup=0.3627 | qwk=0.6936
3/100 * Epoch (train):  38% 638/1692 [03:33<05:27,  3.22it/s, batch_qwk=0.750, loss=0.346, loss_isup=0.346]     Gradient overflow

In [ ]:
train_fold(1)

In [ ]:
train_fold(2)

In [ ]:
train_fold(3)

In [ ]:
train_fold(4)